In [317]:
import imgaug as ia
import numpy as np
from PIL import Image
import cv2 as cv
from imgaug import augmenters as iaa
import numpy as np
from random import randint, uniform, shuffle
import glob
import os
import shutil

In [318]:
def xy_to_yolo_bbox(img, augmented_bbox):
    # x_center y_center width height
    
    abs_width = augmented_bbox.x2 - augmented_bbox.x1
    abs_height = augmented_bbox.y2 - augmented_bbox.y1
    
    x_center = augmented_bbox.x1 + abs_width/2
    y_center = augmented_bbox.y1 + abs_height/2
        
    img_width = img.shape[1]
    img_height = img.shape[0]    
    
    yolo_x = x_center / img_width
    yolo_y = y_center / img_height
    
    yol_w = abs_width / img_width
    yol_h = abs_height / img_height
    
    return yolo_x, yolo_y, yol_w, yol_h


def yolo_bb_to_xy(img, txt_file):
    
    yolo_bboxes = []
    
    with open(txt_file) as f:
        for line in f:
            line = line.strip()
            yolo_bboxes.append([float(x) for x in line.split(' ')[1:]])
    
    img_width = img.shape[1]
    img_height = img.shape[0]
    
    xy_bboxes = []
    
    for bbox in yolo_bboxes:
        x_center = bbox[0] * img_width
        y_center = bbox[1] * img_height

        bbox_width = bbox[2] * img_width
        bbox_height = bbox[3] * img_height

        x1 = x_center - bbox_width/2
        y1 = y_center - bbox_height/2

        x2 = x_center + bbox_width/2
        y2 = y_center + bbox_height/2
    
        
        xy_bboxes.append(ia.BoundingBox(x1=int(x1), y1=int(y1), x2=int(x2), y2=int(y2)))
    
    return ia.BoundingBoxesOnImage(xy_bboxes, shape=img.shape)

In [319]:
def draw_rectangle_augment(img, augmented_bbox):
    
    img = img.copy()
    
    for bbox in augmented_bbox.bounding_boxes:
        cv.rectangle(img, (int(bbox.x1), int(bbox.y1)), (int(bbox.x2), int(bbox.y2)), 1)

    
    return img

In [320]:
images = []
bboxes = []

IMAGES_DIR = 'tt/'

for i in glob.glob(IMAGES_DIR + '/*.jpg'):
    print(i.split('.')[0])
    file_name_no_extension = i.split('.')[0]
    
    img = cv.imread(file_name_no_extension + '.jpg')
    
    bboxes.append(yolo_bb_to_xy(img, file_name_no_extension + '.txt'))
    images.append(img)

tt/2018-10-19-222257_4
tt/2018-10-19-222440_19
tt/2018-10-19-222242
tt/WIN_20181019_22_19_38_Pro
tt/2018-10-19-222257_10
tt/2018-10-19-222254
tt/2018-10-19-222252
tt/2018-10-19-222257_3
tt/2018-10-19-222258
tt/2018-10-19-222440_5
tt/2018-10-19-222235
tt/2018-10-19-222257_14
tt/2018-10-19-222227
tt/2018-10-19-222440_7
tt/2018-10-19-222300
tt/2018-10-19-222236
tt/WIN_20181019_22_20_08_Pro
tt/2018-10-19-222228
tt/2018-10-19-222046
tt/2018-10-19-222440_8
tt/WIN_20181019_22_20_10_Pro
tt/2018-10-19-222257_11
tt/2018-10-19-222440_17
tt/2018-10-19-222211
tt/2018-10-19-222303
tt/2018-10-19-222440_9
tt/2018-10-19-222257_5
tt/2018-10-19-222257_13
tt/WIN_20181019_22_19_40_Pro
tt/2018-10-19-222255
tt/WIN_20181019_22_19_30_Pro
tt/2018-10-19-222440_4
tt/2018-10-19-222241
tt/2018-10-19-222257_7
tt/WIN_20181019_22_19_34_Pro
tt/2018-10-19-222257_9
tt/WIN_20181019_22_19_28_Pro
tt/2018-10-19-222247
tt/2018-10-19-222240
tt/2018-10-19-222257_17
tt/2018-10-19-222300 (1)
tt/2018-10-19-222219
tt/2018-10-19-222

In [321]:
def display(img):
    return Image.fromarray(img[:, :, ::-1])

def Add(seq_l):
    val = randint(-45, 45)
    seq_l.append(iaa.Add(val))

def GaussianBlur(seq_l):
    val = uniform(0.0, 2.0)
    seq_l.append(iaa.GaussianBlur(val))

def Hue(seq_l):
    val = randint(-10,10)
    seq_l.append(iaa.AddToHueAndSaturation(val))

def Noise(seq_l):
    val1 = uniform(0.0, .04)
    seq_l.append(iaa.AdditiveGaussianNoise(scale=val1*225))

def Contrast(seq_l):
    val1 = uniform(.4, 1.2)
    val2 = uniform(.6, 1.4)
    seq_l.append(iaa.ContrastNormalization(alpha=(val1, val2)))


def Perp(seq_l):
    val1 = uniform(.01, .075)
    seq_l.append(iaa.PerspectiveTransform(scale=val1))
    
    
def Crop(seq_l):
    top = randint(0,10)
    right = randint(0,10)
    bottom = randint(0,10)
    left = randint(0,10)
    
def Zoom(seq_l):
    val = uniform(1.0, 1.5)
    seq_l.append(iaa.Affine(scale=val))
    

In [367]:
def augment(images, bboxs, noise, add, blur, hue, contrast, crop, zoom, perp):
    seq_l = []
    
    if noise:
        Noise(seq_l)
    if add:
        Add(seq_l)
    if blur:
        GaussianBlur(seq_l)
    if hue:
        Hue(seq_l)
    if contrast:
        Contrast(seq_l)
    if crop:
        Crop(seq_l)
    if zoom:
        Zoom(seq_l)

    
    seq_l.append(iaa.Fliplr(.5))
    
    seq = iaa.Sequential(seq_l)
    
    seq_det = seq.to_deterministic()
    
    augmented_images = seq_det.augment_images(images)
    

    augmented_bboxes = seq_det.augment_bounding_boxes(bboxs)
    
    print('mug')

    
    print("amount of augmentations: " + str(len(seq_l)))

    return augmented_images, augmented_bboxes

In [368]:
def generate_data(images, bboxs):
    
    result_images = []
    result_bboxes = []
    
    for i in range(1):
        augmented_images, augmented_bboxes = augment(images, bboxes, contrast=0, hue=0, noise=0, add=0,blur=0, zoom=0, crop=0, perp=1)
        
        result_images += augmented_images
        result_bboxes += augmented_bboxes
    
    return result_images, result_bboxes

In [369]:
augmented_images, augmented_bboxes = generate_data(images, bboxes)

TypeError: 'NoneType' object is not subscriptable

In [316]:
AUGMENT_DIR_NAME = 'augmented_data'

if os.path.isdir(AUGMENT_DIR_NAME):
    shutil.rmtree(AUGMENT_DIR_NAME)
else:
    os.makedirs(AUGMENT_DIR_NAME)

for img_num, data in enumerate(zip(augmented_images, augmented_bboxes)):
    aug_img, aug_bbox = data
    
    fp = AUGMENT_DIR_NAME + '/aug' + str(img_num) + '.jpg'
    cv.imwrite(fp, aug_img)
    
    fp = AUGMENT_DIR_NAME + '/aug' + str(img_num) + '.txt'

    with open(fp, mode='w+') as f:
        for cur_bbox in aug_bbox.bounding_boxes:
            x, y, w, h = xy_to_yolo_bbox(aug_img, cur_bbox)
            
            if x < 0.0 or x > 1.0 or y < 0.0 or y > 1.0 or w < 0.0 or w > 1.0 or h < 0.0 or h > 1.0:
                continue
                
            f.write('0 %f %f %f %f\n' % (x, y ,w , h))

            
TRAIN_DIR = './' + AUGMENT_DIR_NAME

with open('train.txt', mode='w+') as f:
    
    for i in range(len(augmented_images)):
        f.write(AUGMENT_DIR_NAME + '/aug' + str(i) + '.jpg\n')  